In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection("mongoProjectDB", "companies")

In [4]:
rest = list(coll.find({"offices.country_code":"USA"}))


In [5]:
def getLocation(company):
    location_empty = []
    for i in range(len(company["offices"])):
        longitude = company['offices'][i]['longitude']
        latitude = company['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location_empty.append(loc)
    return location_empty

In [6]:
for company in rest:
    value = {"$set": {'location':getLocation(company)}}
    coll.update_one(company,value)

In [7]:
for company in rest:
    value = {"$unset": {'offices.latitude':''}}
    value = {"$unset": {'offices.longitude':''}}
    coll.update_one(company,value)

In [8]:
pipeline = [{"$unwind":"$offices"},{"$match":{"$and":[{"offices.country_code":"USA"},{'$or':[{'offices.latitude':{'$ne':None}},{'offices.longitude':{'$ne':None}}]},{"funding_rounds.raised_amount":{"$gte":1000000}},{"deadpooled_year":None}]}}]

In [9]:
query = list(coll.aggregate(pipeline))

In [10]:
df_filtered = pd.DataFrame(query)

In [11]:
data = df_filtered["offices"]

In [12]:
latitude = []
for key, value in data.items():
    latitude.append(value["latitude"])
latitude

[34.090368,
 37.764726,
 37.48413,
 33.022176,
 37.451151,
 37.775196,
 37.789634,
 40.757929,
 40.7464969,
 37.387845,
 37.41605,
 40.7557162,
 37.7768052,
 37.778613,
 37.779558,
 34.017606,
 32.902266,
 37.788482,
 37.295005,
 34.1494713,
 41.885619,
 37.437328,
 37.480999,
 38.926172,
 40.724604,
 34.0263021,
 42.557958,
 40.72604,
 33.429859,
 37.78976,
 37.4233896,
 40.726155,
 40.801358,
 37.392936,
 40.722655,
 37.627971,
 37.465645,
 37.44296,
 47.617682,
 37.488336,
 37.559287,
 37.780716,
 37.269175,
 42.375392,
 37.780134,
 37.757758,
 37.421621,
 37.590339,
 37.7870842,
 47.599925,
 37.374998,
 37.786183,
 47.620982,
 37.408802,
 37.484619,
 37.758113,
 40.7516611,
 33.991101,
 37.7891086,
 37.782103,
 37.09024,
 41.889474,
 37.762681,
 37.5108541,
 37.391502,
 37.796396,
 47.618599,
 37.393679,
 37.418531,
 37.437328,
 37.779507,
 47.615313,
 42.350571,
 37.561877,
 37.40047,
 37.382162,
 37.870744,
 37.775196,
 40.7143528,
 37.425801,
 37.553121,
 37.775196,
 32.872081,


In [13]:
df_filtered["offices"][0]

{'description': 'Headquarters',
 'address1': '9229 W. Sunset Blvd.',
 'address2': '',
 'zip_code': '90069',
 'city': 'West Hollywood',
 'state_code': 'CA',
 'country_code': 'USA',
 'latitude': 34.090368,
 'longitude': -118.393064}

In [14]:
cities_df = []
for e in df_filtered["offices"]:
    cities_df.append(e["city"])

In [15]:
cities_df = set(cities_df)

In [16]:
cities_df.remove("")
cities_df.remove(None)

In [17]:
cities_df = list(cities_df)

In [26]:
len(cities_df)

433

In [27]:
434 * 50

21700

In [28]:
cities_df.remove("Tartu")

ValueError: list.remove(x): x not in list

In [29]:
%store cities_df
len(cities_df)

Stored 'cities_df' (list)


433